In [4]:
import pandas as pd
import numpy as np
import time
import datetime
import os
from pprint import pprint
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper

# logger.debug('Testing')
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

from nanoHUB.clustering.infra import get_mike_only_clustered_users, get_xufeng_only_clustered_users, get_mike_xufeng_clustered_users
from nanoHUB.clustering.infra import create_clusters_repository, add_cluster_info, get_all_clustered_users, get_all_self_study_users, get_all_self_identified_users
from nanoHUB.dataaccess.lake import new_session_lifetime_calculator
application = Application.get_instance('DEBUG')


Function below gives the output for a specific user in the form of table/dataframe

The functio takes the input of a username, startend, enddate in format ("username", "2020-01-01", "2020-12-31") 

The output table provides the name of the user, tool name, first day of the toolstart for that specific toolname under first_day, lastday of the toolstart for the specific tool as last_day. 

The number_toolstart shows the total number of tool_start during the time frame provided.

The number_days provides the total number of  distinct days for which the user has a toolstart ( for example in two weeks I only used the tool on 4 specific days the number_days would output 4)




In [5]:
def user_tool_report(username, startdate, enddate):
    """ Provided a username and start,end date
    returns user's tool report"""
    _user = username
    _startdate = startdate
    _enddate = enddate
    sql  = """
    SELECT 
        user,
        versions.toolname, 
        MIN(DATE(datetime)) AS first_day, 
        MAX(DATE(datetime)) AS last_day, 
        ip, 
        COUNT(toolname) AS total_number_toolstart , 
        COUNT(DISTINCT DATE(datetime)) AS number_Days 
    FROM nanohub_metrics.toolstart 
    INNER JOIN nanohub.jos_tool_version versions 
        ON versions.instance =tool
    WHERE user = "%s"
        AND (DATE(toolstart.datetime) BETWEEN "%s" AND "%s")
    GROUP BY toolname
    ORDER by toolname , datetime
    """ % (_user,_startdate, _enddate)
    users_df =  pd.read_sql_query(sql, nanohub_metrics_db)
    display(users_df)

In [3]:
user_tool_report('gekco', '2021-07-01','2021-12-21')

,user,toolname,first_day,last_day,ip,total_number_toolstart,number_Days
0,gekco,1dhetero,2021-12-01,2021-12-01,128.46.94.233,2,1
1,gekco,abacus,2021-07-23,2021-12-20,173.198.71.146,58,16
2,gekco,aqme,2021-11-04,2021-11-04,128.210.107.129,1,1
3,gekco,atcadlab,2021-11-19,2021-11-21,69.174.156.140,2,2
4,gekco,bandstrlab,2021-11-07,2021-12-16,69.174.156.140,34,2
5,gekco,brillouin,2021-12-01,2021-12-01,128.46.94.233,7,1
6,gekco,cntbte,2021-10-16,2021-10-16,69.174.156.140,5,1
7,gekco,crystalviewer,2021-07-23,2021-12-17,173.198.71.146,36,10
8,gekco,crystal_viewer,2021-10-21,2021-11-09,132.249.202.80,12,2
9,gekco,deviceelectron,2021-07-07,2021-07-07,69.174.156.140,1,1
